# Statistics and Analysis of Characters and Anime 

In [1]:
#Import libraries
import pandas as pd
import numpy as np
import datetime
import ast

from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.preprocessing import MinMaxScaler

import matplotlib.cm as cm
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap


%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 6)
plt.style.use('ggplot')

In [2]:
# Characters who appear >10 times in the fanfiction, their names gathered into a in a csv file. Converted to pandas DF.
characters = pd.read_csv('myhero_names_rev.csv')

# Archive of Our Own
AO3 = pd.read_csv('AO3.csv')

# Series overview: Original season number, Number of episodes, First aired, and Last aired.
og_season_anime_airdate = pd.read_csv('og_air_clean.csv')

# Season 1 Episode number overall, Episode number in the season, Title, Storyboarded by, Directed by,
#     Written by, Original air date, and English air date.
season_one_anime = pd.read_csv('season_one_clean.csv')

# Season 2-6 Episode number overall, Episode number in the season, Title, Storyboarded by, Directed by,
#     Written by, Viewership rating, Original air date, and English air date.
season_two_anime = pd.read_csv('season_two_clean.csv')
season_three_anime = pd.read_csv('season_three_clean.csv')
season_four_anime = pd.read_csv('season_four_clean.csv')
season_five_anime = pd.read_csv('season_five_clean.csv')
season_six_anime = pd.read_csv('season_six_clean.csv')

# Original Video Animation (OVA) titled "Save! Rescue Training!"
# Number (8 total released), Title, Storyboard by, Directed by, Written by, Original release date.
og_vid_animation_anime =  pd.read_csv('OVAs_clean.csv')

# Japan Home Video Release
# Season, Volume, Epsiodes, Release date.
Japan_homevid_release_anime = pd.read_csv('Japan_HV_clean.csv')

# Funimation/Crunchyroll (North America)
# Season, Volume, Episodes, Release date.
funimation_crunchyroll_anime = pd.read_csv('Fun_Crunch_NA_clean.csv')

# Universal Pictures/Manga Entertainment (UK and Ireland)
# Season, Volume, Episodes, Release date.
universalpictures_manga_anime = pd.read_csv('Up_Uk_clean.csv')

# Universal Sony/Madman Entertainment (Australia and New Zealand)
# Season, Volume, Episodes, Release date Universal, Release date Madman.
universalsony_madman_anime = pd.read_csv('Us_Me_clean.csv')

In [3]:
name_1 = [x for x in characters['1'].values]
name_2 = [x for x in (characters['2'].dropna().values)]
mhanames = name_1 + name_2


In [4]:
dont_want = ['Parents', 'Suffering', 'Loss Of His Will', 'Niece', 'mentioned', 'Family', 'Notebooks', 'Character', 'quirk', 'Friend', 'Eriks', 'Werewolf', 'Prototype', 'Mother', 'Sibling', 'Sidekicks', 'l Jade', 'Nezuko']

In [5]:
AO3['ships'] = AO3['ships'].apply(ast.literal_eval)

In [6]:
Ships = AO3.explode('ships')['ships'].value_counts()

In [7]:
characters['First'] = characters['1']
characters = characters.set_index('First')

In [8]:
def pullship(char1, char2):
    name = []
    name2 = []
    try:
        for i in characters.loc[char1]:
            if type(i) != float:
                name.append(i)
    except:
        name.append(char1)
        
    try:
        for i in characters.loc[char2]:
            if type(i) != float:
                name2.append(i)
    except:
        name2.append(char2)
    ship = [name, name2]
    shipname = char1 + ' & ' + char2
    AO3[shipname] = 0
    for index, row in AO3.iterrows():
        for name in ship[0]:
            if row[shipname] == 0:
                if name in ''.join(row['ships']):
                    for name2 in ship[1]:
                        if row[shipname] == 0:
                            if name2 in ''.join(row['ships']):
                                AO3.loc[index, shipname] = 1
                                try:
                                    AO3.loc[index, char1] += 1
                                except:
                                    AO3[char1] = 0
                                    AO3.loc[index, char1] += 1
                                try:
                                    AO3.loc[index, char2] += 1
                                except:
                                    AO3[char2] = 0
                                    AO3.loc[index, char2] += 1


In [9]:
pullship('Deku', 'Katsuki')
pullship('Aizawa', 'Yamada Hizashi')
pullship('Aizawa', 'Deku')
pullship('Katsuki', 'Enjiro')
pullship('Deku', 'Todoroki Sho')
pullship('Dabi', 'Keigo')
pullship('Deku', 'Uraraka')
pullship('Deku', 'All Might')
pullship('Deku', 'Class')
pullship('Denki', 'Shinsou Hitoshi')
pullship('Deku', 'Shinsou Hitoshi')
pullship('Katsuki', 'Reader')
pullship('Katsuki', 'Todoroki Sho')
pullship('Deku', 'Yamada Hizashi')
pullship('Mina', 'Eijirou')
pullship('Jirou Kyouka', 'Denki')
pullship('Deku', 'Shigaraki Tomura')
pullship('Aizawa', 'Shinsou Hitoshi')
pullship('Deku', 'Nedzu')
pullship('Deku', 'Reader')

In [10]:
AO3 = AO3.rename(columns = {'Aizawa & Shinsou Hitoshi': 'Aizawa & Shinsou', 'Shinsou Hitoshi': 'Shinsou',
                            'Jirou Kyouka & Denki': 'Jirou & Denki','Jirou Kyouka': 'Jirou',
                            'Mina & Eijirou': 'Mina & Kirashima', 'Eijirou': 'Kirashima',
                            'Deku & Yamada Hizashi': 'Deku & Prez Mic', 'Yamada Hizashi': 'Prez Mic',
                            'Katsuki & Todoroki Sho': 'Bakugo & Shoto',
                            'Todoroki Sho': 'Shoto', 'Katsuki & Reader': 'Bakugo and Reader', 
                            'Katsuki': 'Bakugo', 'Deku & Shinsou Hitoshi': 'Deku & Shinsou',
                            'Denki & Shinsou Hitoshi': 'Denki & Shinsou', 'Dabi & Keigo': 'Dabi & Hawks',
                            'Keigo': 'Hawks', 'Deku & Todoroki Sho': 'Deku & Shoto',
                            'Katsuki & Enjiro': 'Bakugo & Kirishima', 'Deku & Katsuki': 'Deku & Bakugo',
                            'Aizawa & Yamada Hizashi': 'Aizawa & Prez Mic'})


In [11]:
AO3[1998:].explode('ships')

,title,mixed fandom,ships,characters,hits,kudos,published,Deku & Bakugo,Deku,Bakugo,...,Mina,Kirashima,Jirou & Denki,Jirou,Deku & Shigaraki Tomura,Shigaraki Tomura,Aizawa & Shinsou,Deku & Nedzu,Nedzu,Deku & Reader
1998,Boku No Hero Academia: Ones Way,['Fandoms:\n僕のヒーローアカデミア | Boku no Hero Academi...,NaN,"['Mia Ikati', 'Rue Achlys', 'Ryuuji Takao', 'I...",44,NaN,28 Mar 2023,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1999,Up the Anti: My Hero Academia New Generation,['Fandoms:\n僕のヒーローアカデミア | Boku no Hero Academi...,Midoriya Izuku/Sero Hanta,"['Midoriya Izuku', 'Sero Hanta', 'Bakugou Kats...",28,NaN,28 Mar 2023,1,3,3,...,0,0,0,0,0,0,0,0,0,0
1999,Up the Anti: My Hero Academia New Generation,['Fandoms:\n僕のヒーローアカデミア | Boku no Hero Academi...,Bakugou Katsuki/Kirishima Eijirou,"['Midoriya Izuku', 'Sero Hanta', 'Bakugou Kats...",28,NaN,28 Mar 2023,1,3,3,...,0,0,0,0,0,0,0,0,0,0
1999,Up the Anti: My Hero Academia New Generation,['Fandoms:\n僕のヒーローアカデミア | Boku no Hero Academi...,Iida Tenya/Uraraka Ochako,"['Midoriya Izuku', 'Sero Hanta', 'Bakugou Kats...",28,NaN,28 Mar 2023,1,3,3,...,0,0,0,0,0,0,0,0,0,0
1999,Up the Anti: My Hero Academia New Generation,['Fandoms:\n僕のヒーローアカデミア | Boku no Hero Academi...,Todoroki Shouto/Yaoyorozu Momo,"['Midoriya Izuku', 'Sero Hanta', 'Bakugou Kats...",28,NaN,28 Mar 2023,1,3,3,...,0,0,0,0,0,0,0,0,0,0
